<a href="https://colab.research.google.com/github/1830668NabilahOshin/Numerical-methods-and-Neural-Network-Labwork/blob/main/word2vec%2BBinary_classfication(Multimodels).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Word to Vec

An algorithm which grabs the relational values of a word with its neighboring words from a given input sequence by capturing the meaning and its features on a numerical vector. This vector allows the model to compare the input words and its sequence to grasp similarities, dissimilatiries and connections in a way that is understandable by the model as it has a meaningful numerical representation

In [ ]:
import gensim.downloader as api
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.tree import DecisionTreeClassifier

#Loading pretrained model

In [ ]:
w2v_model = api.load('word2vec-google-news-300')




[==================================================] 100.0% 1662.8/1662.8MB downloaded


# Load dataset in CSV format

In [ ]:

data = pd.read_csv('restuarents.csv', encoding='ISO-8859-1')
print(data.head())

                                                text  label
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1


#word2vec

In [ ]:
tokenized_data = [text.split() for text in data['text']]

vectors = []
for tokens in tokenized_data:
    vec = [w2v_model[token] for token in tokens if token in w2v_model]
    if vec:
        vectors.append(sum(vec) / len(vec))
    else:
        vectors.append(np.zeros(w2v_model.vector_size))

# DataFrame of Word2Vec vectors
w2v_data = pd.DataFrame([x for x in vectors if isinstance(x, np.ndarray)])

w2v_df = pd.DataFrame({'text': data['text'], 'label': data['label'], 'vector': vectors}).dropna()
X_train, X_test, y_train, y_test = train_test_split(w2v_data, w2v_df['label'], test_size=0.2, random_state=42)

print(w2v_df.head())

                                                text  label  \
0                           Wow... Loved this place.      1   
1                                 Crust is not good.      0   
2          Not tasty and the texture was just nasty.      0   
3  Stopped by during the late May bank holiday of...      1   
4  The selection on the menu was great and so wer...      1   

                                              vector  
0  [0.033691406, 0.008544922, 0.06225586, 0.14697...  
1  [0.054270428, -0.098470055, 0.10380045, 0.1255...  
2  [0.039611816, 0.04481252, 0.050313313, 0.06852...  
3  [0.062037878, 0.013691493, 0.015267508, 0.0682...  
4  [-0.0375, 0.11079712, 0.033346556, 0.06557617,...  


#Classified by SVM

In [ ]:
clf = SVC(kernel='linear')


clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
w2v_df['predicted_label'] = clf.predict(w2v_data)

accuracy_w2v_svm = accuracy_score(y_test, y_pred)
print('Accuracy of SVM:', accuracy_w2v_svm)


print(w2v_df.head())


Accuracy of SVM: 0.7431693989071039
                                                text  label  \
0                           Wow... Loved this place.      1   
1                                 Crust is not good.      0   
2          Not tasty and the texture was just nasty.      0   
3  Stopped by during the late May bank holiday of...      1   
4  The selection on the menu was great and so wer...      1   

                                              vector  predicted_label  
0  [0.033691406, 0.008544922, 0.06225586, 0.14697...                1  
1  [0.054270428, -0.098470055, 0.10380045, 0.1255...                0  
2  [0.039611816, 0.04481252, 0.050313313, 0.06852...                0  
3  [0.062037878, 0.013691493, 0.015267508, 0.0682...                0  
4  [-0.0375, 0.11079712, 0.033346556, 0.06557617,...                1  


#Classified by Decision Tree

In [ ]:
clf2 = DecisionTreeClassifier()
clf2.fit(X_train, y_train)


y2_pred = clf2.predict(X_test)

accuracy_w2v_dt = accuracy_score(y_test, y2_pred)
print('Accuracy of dt:', accuracy_w2v_dt)

Accuracy of dt: 0.6065573770491803


#Classified by Logistic Regression

In [ ]:
clf3 = LogisticRegression()
clf3.fit(X_train, y_train)

y3_pred = clf3.predict(X_test)
accuracy_w2v_lr = accuracy_score(y_test, y3_pred)
print('Accuracy of lr:', accuracy_w2v_lr)

Accuracy of lr: 0.7814207650273224


#TF-IDF

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer()

X1_train_tfidf = tfidf_vectorizer.fit_transform(X1_train.astype(str))
X1_test_tfidf = tfidf_vectorizer.transform(X1_test.astype(str))



#Classified by SVM

In [ ]:
clf.fit(X1_train_tfidf, y1_train)

y4_pred = clf.predict(X1_test_tfidf)

accuracy_tfidf_svm = accuracy_score(y1_test, y4_pred)
print('Accuracy of SVM:', accuracy_tfidf_svm)

results_df = pd.DataFrame({
    'text': X1_test,
    'true_label': y1_test,
    'predicted_label': y4_pred
})
print(results_df.head())

Accuracy of SVM: 0.819672131147541
                                                  text  true_label  \
380                           What a mistake that was!           0   
853  It also took her forever to bring us the check...           0   
355  Kids pizza is always a hit too with lots of gr...           1   
357  Cooked to perfection and the service was impec...           1   
362  Today was my first taste of a Buldogis Gourmet...           1   

     predicted_label  
380                0  
853                1  
355                1  
357                1  
362                0  


#Classified by Decision Tree

In [ ]:
clf2.fit(X1_train_tfidf, y1_train)

y5_pred = clf2.predict(X1_test_tfidf)

accuracy_tfidf_dt = accuracy_score(y1_test, y5_pred)
print('Accuracy of DT:', accuracy_tfidf_dt)

Accuracy of DT: 0.7595628415300546


#Classified by Logistic Regression

In [ ]:
clf3.fit(X1_train_tfidf, y1_train)

y6_pred = clf3.predict(X1_test_tfidf)

accuracy_tfidf_lr = accuracy_score(y1_test, y6_pred)
print('Accuracy of lr:', accuracy_tfidf_lr)

Accuracy of lr: 0.7868852459016393
